In [1]:
!pip install pystan==2.19.1.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.2/16.2 MB 55.4 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pystan: filename=pystan-2.19.1.1-cp310-cp310-linux_x86_64.whl size=29422067 sha256=ae1e2e03869d2e84ce94ceb565727d87408de885a2f281fb23896f43c785f73b
  Stored in directory: /root/.cache/pip/wheels/3d/1c/94/4516243362eedbedad15ac4389691ee3bf2d45bec2639c9d8b
Successfully built pystan


In [2]:
import pystan

In [3]:
import warnings
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import seaborn as sns


In [4]:
!git clone --depth=1 https://github.com/hagijyun/tsbook.git

Cloning into 'tsbook'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 39 (delta 6), reused 34 (delta 5), pack-reused 0
Unpacking objects: 100% (39/39), 563.33 KiB | 4.30 MiB/s, done.


In [5]:
code10_1="""
//モデル：規定【ローカルレベルモデル、パラメータが既知】

data{
  int<lower=1> t_max;
  vector[t_max] y;
  
  cov_matrix[1] W;
  cov_matrix[1] V;
  real         m0;
  cov_matrix[1] C0;
}

parameters{
  real  x0;
  vector[t_max] x;
}

model{
  // 尤度の部分
  /* 観測方程式・・・(5.11)式を参照 */
  for (t in 1:t_max){
    y[t] ~ normal(x[t], sqrt(V[1,1]));
}
   // 事前分布の部分
   /* 状態の事前分布 */
   x0 ~ normal(m0, sqrt(C0[1,1]));
   
   /* 状態方程式・・・(5.10)式を参照 */
   x[1] ~ normal(x0, sqrt(W[1,1]));
   for (t in 2:t_max){
     x[t] ~ normal(x[t-1], sqrt(W[1,1]));
   }
}

"""

In [6]:
code10_3="""
//モデル：規定【ローカルレベルモデル、パラメータが未知】

data{
  int<lower=1> t_max;
  vector[t_max] y;
  
  real         m0;
  cov_matrix[1] C0;
}

parameters{
  real  x0;
  vector[t_max] x;
  
  cov_matrix[1] W;
  cov_matrix[1] V;
}

model{
  // 尤度の部分
  /* 観測方程式・・・(5.11)式を参照 */
  for (t in 1:t_max){
    y[t] ~ normal(x[t], sqrt(V[1,1]));
}
   // 事前分布の部分
   /* 状態の事前分布 */
   x0 ~ normal(m0, sqrt(C0[1,1]));
   
   /* 状態方程式・・・(5.10)式を参照 */
   x[1] ~ normal(x0, sqrt(W[1,1]));
   for (t in 2:t_max){
     x[t] ~ normal(x[t-1], sqrt(W[1,1]));
   }
}

"""

In [7]:
code10_5="""
//モデル：規定【ローカルレベルモデル、パラメータが未知、カルマンフィルタを活用】

data{
  int<lower=1> t_max;
  vector[t_max] y;
  
  matrix[1,1] G;
  matrix[1,1] F;
  real         m0;
  cov_matrix[1] C0;
}

parameters{
  cov_matrix[1] W;
  cov_matrix[1] V;
}

model{
  // 尤度の部分
  /* 観測方程式・・・(5.11)式を参照 */
 y ~ gaussian_dlm_obs(F, G, V, W, m0, C0);
 
   // 事前分布の部分
}

"""